In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [35]:
url_inicial = 'https://veterinariamexico.fmvz.unam.mx/index.php/vet/issue/archive/4'
url_root = 'https://veterinariamexico.fmvz.unam.mx/index.php/vet/issue/archive/4'
r=requests.get(url_inicial)

In [36]:
soup = BeautifulSoup(r.text, 'html.parser')

In [37]:
box = soup.find('div', class_='issues media-list')
volumen=box.findAll('div', class_='issue-summary media')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['https://veterinariamexico.fmvz.unam.mx/index.php/vet/issue/view/7',
 'https://veterinariamexico.fmvz.unam.mx/index.php/vet/issue/view/8',
 'https://veterinariamexico.fmvz.unam.mx/index.php/vet/issue/view/1',
 'https://veterinariamexico.fmvz.unam.mx/index.php/vet/issue/view/2',
 'https://veterinariamexico.fmvz.unam.mx/index.php/vet/issue/view/3',
 'https://veterinariamexico.fmvz.unam.mx/index.php/vet/issue/view/4']

In [38]:
def get_url_items(sopa, url): 
    box = soup.find('div', class_='issues media-list')
    volumen=box.findAll('div', class_='issue-summary media')
    vol = [x.find('a').get('href')for x in volumen]
    vol=[urljoin(url_root,i) for i in vol]
    return vol

In [39]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 

Estoy en la pagina https://veterinariamexico.fmvz.unam.mx/index.php/vet/issue/archive/4


In [40]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

6

In [41]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [42]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Veterinaria México OA'
    a='VI. Biotecnología y Ciencias Agropecuarias'
    tem='Ciencias Veterinarias y de la Producción Animal'
    url='http://veterinariamexico.unam.mx'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo de revista']=tire
    except AttributeError:
        content_book['Titulo de revista']=None
    #area
    try:
        area=a
        content_book['Área']=a
    except AttributeError:
        content_book['Área']=None
    #tematica
    try:
        tema=tem
        content_book['Temática']=tem
    except AttributeError:
        content_book['Temática']=None
    #titulo libro
    try:
        titu=s_item.find('h3', class_='media-heading').get_text(strip=True)
        content_book['Titulo del libro']=titu
    except AttributeError:
        content_book['Titulo del libro']=None
    #resumen
    try:
        resu=s_item.find('a', class_='cover').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #Link incial
    try:
        inicial=url
        content_book['Link inicial']=url
    except AttributeError:
        content_book['Link inicial']=None
    #link articulo
    try:
        link=s_item.find('a', class_='galley-link btn btn-primary pdf').get('href')
        content_book['Link articulo']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link articulo']=None
    return content_book

In [43]:
list_scraper=list_scraper[0:6]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5


In [44]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo de revista,Área,Temática,Titulo del libro,Resumen,Link inicial,Link articulo
0,Veterinaria México OA,VI. Biotecnología y Ciencias Agropecuarias,Ciencias Veterinarias y de la Producción Animal,Alteraciones en ovarios de perras por inmuniza...,,http://veterinariamexico.unam.mx,None
1,Veterinaria México OA,VI. Biotecnología y Ciencias Agropecuarias,Ciencias Veterinarias y de la Producción Animal,Valores bioquímicos en sangre de burros antes ...,,http://veterinariamexico.unam.mx,None
2,Veterinaria México OA,VI. Biotecnología y Ciencias Agropecuarias,Ciencias Veterinarias y de la Producción Animal,Evaluación de cuatro vehículos para formular u...,,http://veterinariamexico.unam.mx,None
3,Veterinaria México OA,VI. Biotecnología y Ciencias Agropecuarias,Ciencias Veterinarias y de la Producción Animal,Efectos de la suplementación con quelatos orgá...,,http://veterinariamexico.unam.mx,None
4,Veterinaria México OA,VI. Biotecnología y Ciencias Agropecuarias,Ciencias Veterinarias y de la Producción Animal,Comparación del porcentaje de concepción y la ...,,http://veterinariamexico.unam.mx,None
5,Veterinaria México OA,VI. Biotecnología y Ciencias Agropecuarias,Ciencias Veterinarias y de la Producción Animal,Estudio comparativo de los efectos antivirales...,,http://veterinariamexico.unam.mx,None


In [45]:
df_catalogo.to_csv('Revista57.04.csv', index=False)